In [1]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio
import json
from pprint import pprint
import os

In [2]:
# Use the exact path where the server script was found
server_path = "../market_agents/orchestrators/mcp_server/finance_mcp_server.py"
print(f"Using server script at: {server_path}")

# Verify the file exists
if os.path.exists(server_path):
    print(f"✅ Server script confirmed at: {server_path}")
else:
    print(f"❌ Server script NOT found at: {server_path}")


Using server script at: ../market_agents/orchestrators/mcp_server/finance_mcp_server.py
✅ Server script confirmed at: ../market_agents/orchestrators/mcp_server/finance_mcp_server.py


In [3]:
# Create server parameters with the correct path
server_params = StdioServerParameters(
    command="python",
    args=[server_path],
    env=None
)

In [7]:
# Cell: Initialize connection and list tools with proper ListToolsResult handling
async def initialize_and_list_tools():
    try:
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                # Initialize the connection
                await session.initialize()
                print("Connected to Finance MCP Server")
                
                # List available tools
                tools_result = await session.list_tools()
                print("\nAvailable tools:")
                
                # Print tool information based on the actual structure
                print(f"Tool result type: {type(tools_result)}")
                
                # Handle ListToolsResult object
                if hasattr(tools_result, 'tools'):
                    tool_list = tools_result.tools
                    print(f"Found {len(tool_list)} tools")
                    
                    for tool in tool_list:
                        print(f"\nTool: {tool.name}")
                        print("-" * (len(tool.name) + 6))
                        # Print core attributes
                        print(f"Description: {tool.description}")
                        print("Input Schema:")
                        print(json.dumps(tool.inputSchema, indent=2))
                else:
                    print("No tools attribute found in result")
                    print(f"Available attributes: {dir(tools_result)}")
                
                return tools_result
    except Exception as e:
        print(f"Error in initialize_and_list_tools: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the function to list tools
tools_result = await initialize_and_list_tools()

Connected to Finance MCP Server

Available tools:
Tool result type: <class 'mcp.types.ListToolsResult'>
Found 9 tools

Tool: get_current_stock_price
-----------------------------
Description: 
    Get the current stock price for a given symbol.
    
Input Schema:
{
  "properties": {
    "symbol": {
      "title": "Symbol",
      "type": "string"
    }
  },
  "required": [
    "symbol"
  ],
  "title": "get_current_stock_priceArguments",
  "type": "object"
}

Tool: get_stock_fundamentals
----------------------------
Description: 
    Get fundamental data for a given stock symbol.
    
Input Schema:
{
  "properties": {
    "symbol": {
      "title": "Symbol",
      "type": "string"
    }
  },
  "required": [
    "symbol"
  ],
  "title": "get_stock_fundamentalsArguments",
  "type": "object"
}

Tool: get_financial_statements
------------------------------
Description: 
    Get financial statements for a given stock symbol.
    
Input Schema:
{
  "properties": {
    "symbol": {
      "title"

In [8]:
# Cell: Test calling a tool to verify it's callable
async def test_tool_call():
    try:
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                # Initialize the connection
                await session.initialize()
                print("Connected to Finance MCP Server")
                
                # Test calling the get_current_stock_price tool
                print("\nTesting get_current_stock_price tool:")
                try:
                    result = await session.call_tool(
                        "get_current_stock_price", 
                        arguments={"symbol": "AAPL"}
                    )
                    print(f"Tool call successful!")
                    print(f"Current AAPL price: ${result}")
                    return result
                except Exception as e:
                    print(f"Error calling tool: {type(e).__name__}: {e}")
                    import traceback
                    traceback.print_exc()
                    return None
    except Exception as e:
        print(f"Error in test_tool_call: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test calling a tool
result = await test_tool_call()

Connected to Finance MCP Server

Testing get_current_stock_price tool:
Tool call successful!
Current AAPL price: $meta=None content=[TextContent(type='text', text='235.74', annotations=None)] isError=False


In [9]:
# Cell: Test multiple tools with proper response handling
async def test_multiple_tools():
    try:
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                # Initialize the connection
                await session.initialize()
                print("Connected to Finance MCP Server")
                
                # Test stock price tool
                print("\n1. Testing get_current_stock_price tool:")
                price_result = await session.call_tool(
                    "get_current_stock_price", 
                    arguments={"symbol": "AAPL"}
                )
                print(f"Result type: {type(price_result)}")
                
                # Extract the actual price value
                price_value = price_result
                if hasattr(price_result, 'content') and price_result.content:
                    # If it's a structured response
                    if isinstance(price_result.content, list) and len(price_result.content) > 0:
                        content_item = price_result.content[0]
                        if hasattr(content_item, 'text'):
                            price_value = content_item.text
                
                print(f"Current AAPL price: ${price_value}")
                
                # Test fundamentals tool
                print("\n2. Testing get_stock_fundamentals tool:")
                fundamentals = await session.call_tool(
                    "get_stock_fundamentals", 
                    arguments={"symbol": "MSFT"}
                )
                print(f"MSFT fundamentals result type: {type(fundamentals)}")
                print(f"MSFT fundamentals: {fundamentals}")
                
                # Test company profile tool
                print("\n3. Testing get_company_profile tool:")
                profile = await session.call_tool(
                    "get_company_profile", 
                    arguments={"symbol": "TSLA"}
                )
                print(f"TSLA profile result type: {type(profile)}")
                
                # If it's a large object, just print a few key fields
                if isinstance(profile, dict):
                    key_fields = ['longName', 'sector', 'industry', 'website']
                    selected_profile = {k: profile.get(k) for k in key_fields if k in profile}
                    print("Selected profile information:")
                    for key, value in selected_profile.items():
                        print(f"  {key}: {value}")
                else:
                    print(f"Profile: {profile}")
                
                return {
                    "price": price_value,
                    "fundamentals": fundamentals,
                    "profile": profile
                }
    except Exception as e:
        print(f"Error in test_multiple_tools: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test multiple tools
results = await test_multiple_tools()

Connected to Finance MCP Server

1. Testing get_current_stock_price tool:
Result type: <class 'mcp.types.CallToolResult'>
Current AAPL price: $235.74

2. Testing get_stock_fundamentals tool:
MSFT fundamentals result type: <class 'mcp.types.CallToolResult'>
MSFT fundamentals: meta=None content=[TextContent(type='text', text='{"symbol": "MSFT", "company_name": "Microsoft Corporation", "sector": "Technology", "industry": "Software - Infrastructure", "market_cap": 2981174444032, "pe_ratio": 26.82408, "pb_ratio": 9.850166, "dividend_yield": 0.83, "eps": 12.39, "beta": 0.914, "52_week_high": 468.35, "52_week_low": 381.0}', annotations=None)] isError=False

3. Testing get_company_profile tool:
TSLA profile result type: <class 'mcp.types.CallToolResult'>
Profile: meta=None content=[TextContent(type='text', text='{"address1": "1 Tesla Road", "city": "Austin", "state": "TX", "zip": "78725", "country": "United States", "phone": "512 516 8177", "website": "https://www.tesla.com", "industry": "Auto